In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/ClimaMM/climamm.zip -d /content

In [3]:
# !pip install transformers
# !pip install timm

In [4]:
import glob
from itertools import chain
import os
import random
import zipfile

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, utils
from tqdm.notebook import tqdm

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

import seaborn as sns
import matplotlib.pyplot as plt

# import timm

from time import sleep
import warnings
from transformers import AdamW

from transformers import CLIPProcessor, CLIPModel

In [5]:
BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 1e-3

seed = 510
task= 'stance'
NUM_LABELS = 3

model_used = 'clip_img_only'

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [7]:
class ImageDataset(Dataset):
    def __init__(self, data, img_dir, split, transform=None):
        self.data = data[data['split'] == split].reset_index(drop=True)

        if task == 'target':
            self.data = self.data[self.data['hate'] == 1].reset_index(drop=True)

        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {}
        img_name = os.path.join(self.img_dir, self.data.loc[idx, 'name'])
        label = self.data.loc[idx, task]
        text = self.data.loc[idx, 'text']

        if self.transform:
          image = self.transform(image)

        try:
            sample["label"] = int(label)
            # sample["text"] = text
            sample["text"] = text[:77]
            sample["image"] = img_name
        except Exception as e:
            print(e)

        return sample

In [8]:
csv_path = 'drive/MyDrive/ClimaMM/climamm.csv'
img_dir = '/content/climamm'

data = pd.read_csv(csv_path)

train_dataset = ImageDataset(data=data, img_dir=img_dir, split='train')
val_dataset = ImageDataset(data=data, img_dir=img_dir, split='val')
test_dataset = ImageDataset(data=data, img_dir=img_dir, split='test')

train_data_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_data_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


dataloaders = {'Train': train_data_loader, 'Test': test_data_loader, 'Val': val_data_loader}


# for image, label in train_dataset:
#     pass

In [9]:
print(len(train_data_loader))
print(len(val_data_loader))
print(len(test_data_loader))

425
26
50


In [10]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()

        # self.fc1 = nn.Linear(1792, 128)

        #Image Only
        # self.fc1 = nn.Linear(1024, 128)

        self.fc1 = nn.Linear(768, 128)


        self.fc2 = nn.Linear(128, NUM_LABELS)

        self.dropout = nn.Dropout(0.25)

    def forward(self, input):

      linear_out = self.dropout(F.relu(self.fc1(input)))

      final_out = self.fc2(linear_out)

      return final_out

In [11]:
device = 'cuda'
print(device)

clip = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)

model = Model().to(device)

cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [12]:
#optimizer
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE, eps=1e-8)

processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

#Loss function
criterion = nn.CrossEntropyLoss()


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [13]:
#training
def train():
  train_loss = []
  train_acc = []

  val_loss = []
  val_acc = []

  for epoch in range(0, EPOCHS):
    print('-'*50)
    print('Epoch {}/{}'.format(epoch+1, EPOCHS))

    for phase in ['Train', 'Val']:

      batch_loss = 0.0000   #live loss
      batch_acc = 0.0000   #live accuracy

      y_pred = []
      y_true = []

      if phase == 'Train':
          model.train()
      else:
          model.eval()

      with tqdm(dataloaders[phase], unit="batch", desc=phase) as tepoch:

        for idx, batch in enumerate(tepoch):
          labels = batch["label"].to(device)
          text = batch["text"]

          imgs = []

          img_paths = batch["image"]

          for path in img_paths:

            imgs.append((Image.open(path)))


          ## Multimodal

          # inputs = processor(text=text, images=imgs, return_tensors="pt", padding=True).to(device)

          # clip_output = clip(**inputs)

          # img_embed = clip_output.vision_model_output.pooler_output

          # text_embed = clip_output.text_model_output.pooler_output

          # concat = torch.cat((text_embed, img_embed), 1)

          # output = model(concat)


          # #Image only

          # inputs = processor(images=imgs, return_tensors="pt").to(device)

          # img_embed = clip.get_image_features(**inputs)

          # output = model(img_embed)

          #Text only

          inputs = processor(text=text, images=imgs, return_tensors="pt", padding=True).to(device)

          clip_output = clip(**inputs)

          text_embed = clip_output.text_model_output.pooler_output

          output = model(text_embed)

          loss = criterion(output, labels)

          if phase == 'Train':

              #zero gradients
              optimizer.zero_grad()

              # Backward pass  (calculates the gradients)
              loss.backward()

              optimizer.step()             # Updates the weights

          batch_loss += loss.item()

          _, preds = output.data.max(1)

          y_pred.extend(preds.tolist())
          y_true.extend(labels.tolist())

          tepoch.set_postfix(loss = batch_loss/(idx+1))

          #accumulate loss and accuracy

        if phase == 'Train':
          train_loss.append(batch_loss/(idx+1))
          train_acc.append(batch_acc)

        else:
          val_loss.append(batch_loss/(idx+1))
          val_acc.append(batch_acc)

In [14]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [15]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import torchmetrics

def test():
  model.eval()
  y_pred = []
  y_true = []

  # auroc_list = []

  auroc = torchmetrics.AUROC(task='multiclass', num_classes = NUM_LABELS)

  with tqdm(dataloaders['Test'], unit="batch") as tepoch:
    for idx, batch in enumerate(tepoch):

      labels = batch["label"].to(device)
      text = batch["text"]

      imgs = []

      img_paths = batch["image"]

      for path in img_paths:

        imgs.append((Image.open(path)))

      # Multimodal
      # inputs = processor(text=text, images=imgs, return_tensors="pt", padding=True).to(device)

      # clip_output = clip(**inputs)

      # img_embed = clip_output.vision_model_output.pooler_output

      # text_embed = clip_output.text_model_output.pooler_output

      # concat = torch.cat((text_embed, img_embed), 1)

      # output = model(concat)


      #  #Image only
      # inputs = processor(images=imgs, return_tensors="pt").to(device)

      # img_embed = clip.get_image_features(**inputs)

      # output = model(img_embed)


       #Text only

      inputs = processor(text=text, images=imgs, return_tensors="pt", padding=True).to(device)

      clip_output = clip(**inputs)

      img_embed = clip_output.vision_model_output.pooler_output

      text_embed = clip_output.text_model_output.pooler_output

      output = model(text_embed)




      _, preds = output.data.max(1)

      preds_proxy = torch.sigmoid(output).detach().cpu()

      auroc.update(preds_proxy, batch["label"])

      y_pred.extend(preds.tolist())
      y_true.extend(labels.tolist())

  print("Model: ", model_used )
  print("Seed: ", seed )
  print("Task: ", task )

  print("Accuracy:{:.4f}".format(accuracy_score(y_true, y_pred) ))
  print("Recall:{:.4f}".format(recall_score(y_true, y_pred,average='macro') ))
  print("Precision:{:.4f}".format(precision_score(y_true, y_pred,average='macro') ))
  print("f1_score:{:.4f}".format(f1_score(y_true, y_pred,average='macro') ))

  print("AUC:{:.4f}".format(auroc.compute()))

  print(classification_report(y_true,y_pred))
  print(confusion_matrix(y_pred,y_true))


In [ ]:
train()
test()

--------------------------------------------------
Epoch 1/5


Train:   0%|          | 0/425 [00:00<?, ?batch/s]